Import packages!

In [3]:
import numpy as np

Read the splice data

In [4]:
fname = 'splice.data'
with open(fname) as f:
    content = f.readlines()
content = [x.strip().split(',') for x in content]
for l in content:
    for i in range(len(l)):
        l[i] = l[i].strip()
raw_data = np.array(content)

In [5]:
# Given one 'row' of data, constructs an input and output out of it
def getInstance(row):
    one_hot = np.zeros((1, 3))
    label = row[0]
    if label == 'EI':
        one_hot[0][0] = 1
    elif label == 'IE':
        one_hot[0][1] = 1
    else:
        one_hot[0][2] = 1
    orth = np.zeros((len(row[2]), 4))
    for i in range(len(row[2])):
        c = row[2][i]
        if c == 'A':
            orth[i][0] = 1
        elif c == 'C':
            orth[i][1] = 1
        elif c == 'G':
            orth[i][2] = 1
        elif c == 'T':
            orth[i][3] = 1
        elif c == 'D':
            orth[i][0] = 1/3.
            orth[i][2] = 1/3.
            orth[i][3] = 1/3.
        elif c == 'N':
            orth[i][0] = 1/4.
            orth[i][1] = 1/4.
            orth[i][2] = 1/4.
            orth[i][3] = 1/4.
        elif c == 'S':
            orth[i][1] = 1/2.
            orth[i][2] = 1/2.
        elif c == 'R':
            orth[i][0] = 1/2.
            orth[i][2] = 1/2.
    return orth, one_hot

In [14]:
x = []
y = []
for i in range(raw_data.shape[0]):
    xi, yi = getInstance(raw_data[i])
    x.append(xi)
    y.append(yi)
x = np.array(x)
y = np.array(y)

# Shuffle dataset since it's unevenly distributed
shuffle_index = np.random.permutation(len(x))
x = x[shuffle_index]
y = y[shuffle_index]

# Conv1D expects channels first
x = np.swapaxes(x, 1, 2)
print (x.shape, y.shape)

(3190, 4, 60) (3190, 1, 3)


Construct a classification model

In [7]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv1D, Dropout

Using TensorFlow backend.


In [21]:
model = Sequential()
model.add(Conv1D(16, 3, input_shape=(None, 60), padding='same'))
model.add(Activation('relu'))
model.add(Conv1D(32, 3, padding='same'))
model.add(Activation('relu'))
model.add(Dense(3))
model.add(Dropout(0.5))
model.add(Activation('softmax'))

In [22]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [23]:
model.fit(x, y, epochs=30, batch_size=32, validation_split=0.1, shuffle=True)

Train on 2871 samples, validate on 319 samples
Epoch 1/30
2871/2871 [==============================] - 2s 805us/step - loss: 1.0513 - acc: 0.4046 - val_loss: 1.0045 - val_acc: 0.5321
Epoch 2/30
2871/2871 [==============================] - 1s 191us/step - loss: 0.9783 - acc: 0.4627 - val_loss: 0.8805 - val_acc: 0.6599
Epoch 3/30
2871/2871 [==============================] - 1s 182us/step - loss: 0.8336 - acc: 0.5595 - val_loss: 0.6496 - val_acc: 0.8103
Epoch 4/30
2871/2871 [==============================] - 0s 167us/step - loss: 0.6729 - acc: 0.6323 - val_loss: 0.4647 - val_acc: 0.8636
Epoch 5/30
2871/2871 [==============================] - 0s 158us/step - loss: 0.5616 - acc: 0.6723 - val_loss: 0.3487 - val_acc: 0.8918
Epoch 6/30
2871/2871 [==============================] - 0s 151us/step - loss: 0.5166 - acc: 0.6842 - val_loss: 0.2998 - val_acc: 0.9154
Epoch 7/30
2871/2871 [==============================] - 0s 146us/step - loss: 0.4796 - acc: 0.6964 - val_loss: 0.2554 - val_acc: 0.9216
E